<a href="https://colab.research.google.com/github/mzucali/pyMinColab/blob/main/ExtractFromSummaryEMPA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Marzo 2023 -
LEGGE IL FILE SUMMARY.TXT ed estrae DL e SD:

=>>input il file summary.txt o qualunque altro nome MA .TXT

In [1]:
# load and read summary file from EMPA
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving summary_short.txt to summary_short.txt
User uploaded file "summary_short.txt" with length 43451 bytes


In [2]:
num_elements = input("QUANTI ELEMENTI ANALIZZATI? ")
#num_elements = 11
loop = int(num_elements)+2

QUANTI ELEMENTI ANALIZZATI? 12


In [3]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Mar 14 11:30:45 2023
GRAZIE ALESSANDRO COMUNIAN
@author: alex
"""

#file_in = open("summary.txt", "r")
file_in = open(fn,"r")
outFile = open("result2_tmp.txt", "w")

lines = file_in.readlines()

for i, line in enumerate(lines):
    # Just a spacing line
    line_sp = line.split()
    print(line_sp)
    line_s = line.split("\t")
    print(line_s)
    if line_s:
        try:
            if(line_s[4] == "Comment"):
                # Keep track of the comment value
                comm = line_s[6]
                # Just a spacing line
                print()
        except IndexError:
            pass

        if(line_s[0] == "Curr.(A)"):
#        if(line_s[0] == "Element	"):
            print("ID {0} Comment".format(lines[i+1][:-1]))
            outFile.write("ID {0} Comment\n".format(lines[i+1][:-1]))
            for j in range(2,loop):
                print("{0} {1}\n".format(lines[i+j][:-1], comm), end="")
                outFile.write("{0} {1}\n".format(lines[i+j][:-1], comm))
                #outFile.write(" {0}\n".format(lines))

file_in.close()
outFile.close() #result2.txt



[' \n']
['Intensity & Mass%', '      Group : FabiolaCaso   ', '  Sample : 18-1-22         ', ' Page ', '  1 ', ' \n']
[' \n']
[' \n']
[' \n']
[' \n']
['Unknown Specimen', 'No.', '1', ' \n']
['Group :', 'FabiolaCaso', 'Sample :', '18-1-22', ' \n']
['UNK No. :', '1', 'Comment :', 'Line 1 AS584-S1-L1  \n']
['Stage :', 'X=', '52.8390', 'Y=', '63.5565', 'Z=', '11.1440', ' \n']
['Acc. Voltage :', '15.0', '(kV)', 'Probe Dia. :', '0', 'Scan :', 'Off', ' \n']
['Dated on', 'Jan 18 10:35 2022', ' \n']
['WDS only', 'No. of accumulation :', '1', ' \n']
[' \n']
['Curr.(A) :', '5.003E-09', ' \n']
['Element', 'Peak(mm)', 'Net(cps)', 'Bg-', 'Bg+', 'S.D.(%)', 'D.L.(ppm)', ' \n']
['1', 'Na', '129.793', '0.3', '2.4', '1.3', '163.23', '', '244', ' \n']
['2', 'Al', '90.523', '929.3', '6.0', '6.0', '0.60', '164', ' \n']
['3', 'Ti', '87.989', '0.3', '6.9', '7.2', '267.36', '', '321', ' \n']
['4', 'Mn', '146.043', '19.4', '5.0', '5.9', '5.41', '312', ' \n']
['5', 'K', '119.659', '-1.2', '11.7', '10.8', '100.00

In [4]:
#RESULT3 con spazi e tab corretti
newopen = open("result2_final.txt", "w")
with open("result2_tmp.txt") as f:
  first_line = f.readline()
  first_line_split = first_line.split()
  first_line = '\t'.join(first_line_split)+'\n'
  newopen.write(first_line)
  print(first_line)

with open("result2_tmp.txt") as f:
  for line in f:
    if 'ID Element	Peak(mm)' not in line:
      line_split = line.split()
      line = '\t'.join(line_split)+'\n'
      print(line)
      newopen.write(line)
newopen.close()

RESULT3.TXT con dati estratti di SD e DL

In [5]:
# EXTRACT X and Y coodinates
file_in2 = open(fn, "r")
lines = file_in2.readlines()
names = []
Xs = []
Ys = []
Zs = []

for i, line in enumerate(lines):
    line_s = line.split()
    #print(line_s)
    if line_s:
        try:
            if(line_s[4] == "Comment"):
                # Keep track of the comment value
                name = line_s[6]
                names.append(name)
                # Just a spacing line
            elif(line_s[0] == "Stage"):
                # Keep track of the comment value
                x = line_s[3]
                Xs.append(x)
                y = line_s[5]
                Ys.append(y)
                z = line_s[7]
                Zs.append(z)
                # Just a spacing line
        except IndexError:
            pass

import pandas as pd
# Just a spacing line
print("Sample: ",len(names))
print("X: ", len(Xs))
print("Y: ", len(Ys))
print("Z: ", len(Zs))
out = pd.DataFrame(
    {'Sample': names,
     'X':Xs,
     'Y':Ys,
     'Z':Zs
     })
# Just a spacing line
print(out)
out.to_csv('coordinate.csv', index=False)

Sample:  22
X:  22
Y:  22
Z:  22
   Sample        X        Y        Z
0    Line  52.8390  63.5565  11.1440
1    Line  52.8330  63.7800  11.1430
2    Line  52.8270  63.9580  11.1420
3    Line  52.8210  64.1585  11.1415
4    Line  52.8475  64.3495  11.1405
5    Line  52.7075  64.6045  11.1395
6    Line  52.8035  64.7605  11.1385
7    Line  52.7690  64.9365  11.1380
8    Line  52.8180  65.1880  11.1370
9    Line  52.8055  65.3435  11.1360
10   Line  52.8205  65.5740  11.1350
11   Line  52.7515  65.7705  11.1345
12   Line  52.7345  65.9935  11.1335
13   Line  52.7465  66.2015  11.1325
14   Line  52.7035  66.4370  11.1315
15   Line  52.7405  66.5810  11.1310
16   Line  52.7630  66.7795  11.1300
17   Line  52.7385  66.9675  11.1290
18   Line  52.7175  67.1845  11.1280
19   Line  52.7265  67.3690  11.1275
20   Line  52.7120  67.5485  11.1265
21   Line  52.6595  67.7785  11.1255


COORDINATE.CSV con sample name e coordinate X e Y

TUTTO DA **SCARICARE**